## Problématique sur l'instruction groupby()
***

Pandas est une librairie très efficace en matière de DS mais elle est très lente.<br>
Une 1ère problématique a été relevée pour la lecture des fichiers csv/txt pour les FEC :<br>
Le temps de chargement est extrêment lent compte tenu que l'instruction lit cellule par cellule, <br>
on a eu recours à la librairie PyArrow.<br>
Autre problématique relevée : l'instruction groupby() agit comme une pure boucle 'for' de Python, <br>
et là aussi, un FEC d'une taille de 100Mo, l'instruction groupby() s'exécute au bout de 6 minutes environ.<br>
Il y a une librairie qui peut résoudre cette lenteur pour certaines instructions de Pandas <br>
(Lien : https://www.kdnuggets.com/2019/11/speed-up-pandas-4x.html), mais pas pour GroupBy() qui a <br>
été résolue à partir du programme ci-dessous.<br>

Éditeur : Laurent REYNAUD</br>

In [1]:
# Modules
import pandas as pd
import pyarrow.csv as csv

In [2]:
# Chemin et nom du FEC à charger
my_path = 'C:/Users/LRCOM/pythonProjects/lr_dscf1/123456789FEC20171231.csv'

In [32]:
# Récupération des traitements opérés sur le FEC en matière de DS
# accounting_ledger = ds.Treatments(my_path).my_df
# balance = ds.Treatments(my_path).balance    
# audit = ds.Treatments(my_path).audit

In [3]:
%%time

# Récupération du fichier .csv en DF pyarrow
my_df = csv.read_csv(my_path)

CPU times: total: 219 ms
Wall time: 149 ms


In [4]:
%%time

# Conversion de la DF pyarrow sous format pandas
my_df = my_df.to_pandas()

CPU times: total: 15.6 ms
Wall time: 23.5 ms


<h5> Récupération des champs JournalCode et JournalLib

In [35]:
%%time

# Méthode groupby()
# my_df_sum = my_df.groupby(['JournalCode', 'JournalLib'])['EcritureNum'].sum().reset_index()
# my_df_sum.drop(columns=['EcritureNum'], inplace=True)

CPU times: total: 0 ns
Wall time: 0 ns


In [36]:
# Autre méthode que groupby() détaillée dans les instructions ci-après
df_journal = my_df[['JournalCode', 'JournalLib']].copy()

In [37]:
df_journal['Désignation'] = df_journal['JournalLib'] + '-' + df_journal['JournalCode']

In [38]:
my_array = df_journal['Désignation'].unique()
my_list = my_array.tolist()

In [39]:
code_journal = []
lib_journal = []
for i in my_list:
    code_journal.append(i.split('-', 1)[1])
    lib_journal.append(i.split('-', 1)[0])

In [40]:
new_df = pd.DataFrame({'Code': code_journal, 'Désignation': lib_journal}).sort_values(by='Code')
list_journal = new_df.to_numpy().tolist()
list_journal

[['29', 'DEUTSCHE BANK'],
 ['35', 'CIC'],
 ['60', 'JOURNAL DES ACHATS'],
 ['70', 'JOURNAL DE VENTES'],
 ['80', 'OPERATIONS DIVERSES'],
 ['81', 'OD DE TVA'],
 ['96', 'CHEQUES A ENCAISSER'],
 ['97', 'EFFETS A RECEVOIR'],
 ['AA', 'Ecriture immo.'],
 ['AB', 'Pièce comptable'],
 ['AF', 'Ecritures amort.'],
 ['DA', 'Pièce client'],
 ['DR', 'Facture client'],
 ['DV', 'Avoir client avis'],
 ['FAR', 'FACTURES A RECEVOIR'],
 ['GV', 'Facture GS'],
 ['KA', 'Pièce fournisseur'],
 ['KD', 'Avoir Fournis.'],
 ['KG', 'Note crédit fourn.'],
 ['KN', 'Net fournisseurs'],
 ['KP', 'Gestion de compte'],
 ['KR', 'Facture fournisseur'],
 ['OP', 'Pièce comptable'],
 ['PAIE', 'JOURNAL DE PAIE'],
 ['PR', 'Modification de prix'],
 ['RAN', 'Report A Nouveau'],
 ['RN', 'Facture (net)'],
 ['RS', 'Facture (net)ext.'],
 ['RV', 'Reprise de facture'],
 ['SA', 'Pièce compte général'],
 ['ST', 'extourne'],
 ['WA', 'Sortie marchandises'],
 ['WE', 'Entrée marchandises'],
 ['WI', "Pièce d'inventaire"],
 ['WL', 'Sortie march./

<h5> Récupération d'une classe du PCG

In [7]:
%%time

# Méthode groupby()
accounts = my_df.loc[my_df['Compte'].astype('str').str.contains('^1')]
accounts.groupby('Compte').count().reset_index()

CPU times: total: 15.6 ms
Wall time: 19 ms


,Compte,EcritureNum,JournalCode,EcritureDate,Intitule,EcritureLib,PieceRef,Debit,Credit,EcritureLet,...,PieceDate,ValidDate,Montantdevise,Idevise,DateLet,EcritureDate_time,PieceDate_time,ValidDate_time,DateLet_time,__index_level_0__
0,101300,1,1,1,1,1,1,1,1,1,...,1,1,1,0,1,1,1,1,0,1
1,106110,1,1,1,1,1,1,1,1,1,...,1,1,1,0,1,1,1,1,0,1
2,110000,3,3,3,3,3,3,3,3,3,...,3,3,3,0,3,3,3,3,0,3
3,129000,2,2,2,2,2,2,2,2,2,...,2,2,2,0,2,2,2,2,0,2


In [8]:
%%time

# Autre méthode que groupby() détaillée dans les instructions ci-après
accounts = my_df.loc[my_df['Compte'].astype('str').str.contains('^1')]
accounts['Compte'].unique()

CPU times: total: 0 ns
Wall time: 0 ns


array(['129000', '106110', '110000', '101300'], dtype=object)